In [1]:
import sys
sys.path.append('../src')
import tarski
from tarski.symbols import *

# Functional STRIPS

Functional STRIPS (FSTRIPS, for short) is a classical planning language that extends STRIPS with functional systems in order to provide a number of _expressive_ **and** _computational_ advantages, such as the ability of making indirect refernce to domain objects through the use of nested terms.

```Tarski``` lifts [Geffner's original restrictions]() on the language, and follows a full first-order logic approach. We will use the classical Blocks World domain as a running example.

### Problem Language

```Tarski``` supports the formulation of FSTRIPS problems $P$ defined over a many-sorted first-order logic language with equality. We note the language associated with a prolem $P$ as ${\cal L}(P)$.

As we have seen in previous chapters of this Tutorial, ```Tarski``` languages are made up of a finite set of sorts $T$, a possibly infinite number of variables $v_{1}^t$, $v_{2}^t$,\ldots for each sort $t \in T$, a set $\Phi$ of _function symbols_ and a set $\Pi$ of _relation symbols_, assumed to include a number of symbols with fixed denotation, such as equality.

We can create the language ${\cal L}(P)$ for _Blocks World_ as follows

In [2]:
# 1. Create language to model the domain
bw = tarski.language()

along with types $T_{blocksworld}$,

In [3]:
# 2. Define sorts that organise the objects considered
place = bw.sort('place')
block = bw.sort('block', [place])

domain specific function symbols $\phi_{blocks world}$

In [4]:
# 3. Define functions
loc = bw.function( 'loc', block, place )
width = bw.function('width', place, bw.Real)
looking_at = bw.function( 'looking_at', block )

and domain specific predicate symbols $\Pi_{blocksworld}$

In [5]:
# 4. Define predicates
clear = bw.predicate( 'clear', block)

## Problem Interpretations

The current version of ```Tarski``` requires that the [universe of discourse](https://en.wikipedia.org/wiki/Domain_of_discourse#Universe_of_discourse) in any FSTRIPS problem $P$ is **fixed**. That is, all valid interpretations ${\cal M}$ for ${\cal L}(P)$ have the same universe, denoted by ${\cal U}_P = \{ {\cal U}_t \}_{t \in T}$. It is further required that the universe ${\cal U}_t$ of any sort $t$ is _finite_, this is a limitation of the current and provably future versions of ```Tarski```.

In our model of _Blocks World_ interpretations ${\cal M}$ for ${\cal L}(P)$ consist of two sorts as defined above. We set ${\cal U}_{place}$ to $\{ b_1, b_2, b_3, b_4, table \}$ as follows



In [6]:
table = bw.const('table', place)
b1, b2, b3, b4 = bw.const(('b_{}'.format(k) for k in (1,2,3,4)), place)
the_places = [table, b1, b2, b3, b4]

that is, the set of names of objects we can place other objects on. The universe of the second sort we defined, ${\cal U}_{blocks}$ is set to $\{ b_1, b_2, b_3, b_4\}$ as follows

In [7]:
b1, b2, b3, b4 = bw.const(('b_{}'.format(k) for k in (1,2,3,4)), block)
the_blocks = [b1, b2, b3, b4]

We note that ```the_places[1]``` and ```the_blocks[0]``` are different Python objects

In [8]:
the_places[1].dump()

{'sort': 'place', 'symbol': 'b_1'}

In [9]:
the_blocks[0].dump()

{'sort': 'block', 'symbol': 'b_1'}

with different semantics, which are exposed invoking the method ```dump()``` that returns a dictionary suitable to serialise the object in plain text. Yet, both refer to the same symbol: as is customary in Python, objects are best understood as _handles_ to access stuff, rather than the stuff itself.

As a result of the universe of discourse being fixed, every interpretation for the language ```bw``` shares sorts and associated constant symbols. On the other hand, the denotation of functions and predicates is subject to change. Interpretations (or models) ${\cal M}$ of ```Tarski``` languages are objects that allow to associate with function and predicate symbols their denotation. 

In [10]:
M = bw.model()

For the purposes of this example, we will set the denotation of function and predicate symbols as follows

In [11]:
M.set( loc, (b1, b2)) # loc(b1) := b2
M.set( loc, (b2, b3)) # loc(b2) := b3
M.set( loc, (b3, table)) # loc(b3) := table
M.set( loc, (b4, table)) # loc(b4) := table

M.set( width, (b1, 3.0)) # width(b1) := 3.0
M.set( width, (b2, 2.0)) # width(b2) := 2.0
M.set( width, (b3, 1.0)) # width(b3) := 1.0
M.set( width, (b4, 0.5)) # width(b4) := 0.5

M.set( looking_at, table) # looking_at() := table

M.add( clear, b1) # clear(b1)
M.add( clear, b4) # clear(b4)

## Action Schemas

A Functional STRIPS _action schema_ $a$ is **defined** by:
 - a _name_, an alphanumeric identifier,
 - a _signature_ $(v_1,\ldots,v_n)$, where each $v_i$ is a variable of some sort $T$,
 - a _precondition_ formula $\mathrm{pre}(a)$, an arbitrary formula over ${\cal L}(P)$,
 - a set of _conditional effects_ $\mathrm{effs}(a)$
 
Each effect $e \in \mathrm{effs}(a)$ can be either a _relational_ or _functional_ effect. Actions have _unspecified_ durations, measured as number of _time steps_ $dur(a)$. In Classical FSTRIPS problems durations of all actions are $1$, Differential FSTRIPS problems allow $dur(a)$ to be fixed to or bounded by an arbitrary constant.

### Relational Effects

Relational effects of actions $a$ specify _general_ properties states _must_ have after the action has been executed. The current version of ```Tarski``` requires relational effects to be of the form

$$
\phi \rightarrow L
$$

where $\phi$ is an arbitrary formula over ${\cal L}(P)$ and $L$ is a **literal**. Literals are _atomic formulas_ $R(\bar{t})$ or their negation, $\neg R(\bar{t})$. In the first case, we say that the effect is an _add effect_, in the second, a _delete effect_.


### Functional Effects

Functional effects of actions $a$ allow actions to set _directly_ the denotation of a given functional symbol $f(\bar{t})$ in states resulting from $a$ being done. ```Tarski``` current implementation of Functional STRIPS requires these effects to have the form

$$
\phi \rightarrow f(\bar{t}) := w
$$

where $w$ is an **arbitrary** term. $f(\bar{t})$ is made up of a function symbol $f$, of whatever arity, over a tuple of terms $\bar{t}$ of appropiate size and type. The types of the term $w$ and symbol $f$ must coincide.

### Constructing action schemas with ```Tarski```



In [12]:
import tarski.fstrips

with bw.var('dest', block) as dest :
    tarski.fstrips.action(bw, name='move', \
                         parameters=[dest], \
                         precondition=land(clear(looking_at()), clear(dest), width(looking_at()) < width(dest)),
                         effects = [
                                    neg(clear(dest)), loc(looking_at()) == dest
                                   ])

**Next**: [TBD]()